# InfCG Lab3 

In this lab we will learn how we can improve the speed of computations using [NumPy](http://www.numpy.org/).
NumPy is a Python package for numeric computation that offers optimized routines for vectorized data.

Vectorization, refers to the process of rewriting an iterative program (a program that has loops), such that it has (nearly) no loops. Instead of sequentially performing computations, such a vectorized program performs subsets of operations at once (for trivial tasks all operations might be applied at once). 

In this course, we will use vector-based representations when implementing neuronal nets (and also in assignment 2) - but applying vectorization is a very important and useful concept in all of data-science and machine learning.

As you will see in this Lab, formulating problems in vectorized form can lead to large speed improvements.
Before getting into the details of how to perform computations using NumPy, let's see *how much faster* a vectorized version can be.



## Why Vectorize?

Let's assume that we have a large number of data stored in ```long_list``` and we want to calculate the sum of all elements in it.

In [373]:
long_list = list(range(500000))

We have seen last week that we can easily **loop through** a Python data structure (in this case a list) using loops.
If we want to calculate the sum of all elements in the list we simply store the partial sum while we iterate through the list, like so:

In [374]:
partial_sum = 0

for number in [1,2,3,4,5]:
    partial_sum = partial_sum + number 
    #this sort of operation is performed all the time so there exists a shorthand
    #if you want you can also write partial_sum += number

partial_sum

15

And since this sort of operation might be useful in the future we might want to make it a function:

In [375]:
def my_list_sum(list_of_numbers):
    partial_sum = 0
    for number in list_of_numbers:
        partial_sum += number
    return partial_sum

my_list_sum([1,2,3,4,5])

15

In [376]:
my_list_sum(long_list)

124999750000

This approach is very simple and easy to understand and computing it didn't really take *that long*. Still, if we want to compare implementations we need to calculate how long it took. For that, we can use an IPython [magic command](http://ipython.readthedocs.io/en/stable/interactive/magics.html) 

```
%%timeit
```

``timeit`` calculates the average time it takes to execute a Python expression for a number of runs (and takes care of a number of issues that make estimating processing time tedious). We can use the magic command in a cell like so:

In [1]:
%%timeit
my_list_sum(long_list)

NameError: name 'my_list_sum' is not defined

So as we've said before, it didn't really take long for this list - on a (rather slow) laptop around 31 [milliseconds](https://en.wikipedia.org/wiki/Millisecond).

Summing is a very common operation and of course there exists a build-in operator for summing in Python - we can sum elements of a list using Python's own ``sum()`` implementation.
Let's check how long it takes us to sum the elements using Python's own implementation:

In [378]:
%%timeit
sum(long_list)

100 loops, best of 3: 6.11 ms per loop


On the same laptop our self-made loop takes roughly 5 times longer (your results might vary but the Python version should be considerably faster).

#### Why are built-ins faster?

When we write loops, the code is executed as Python code, i.e. the Python interpreter has to translate the Python code into bytecode instructions. The native Python operations (like ```sum, len```) are all written as optimized code and thus do not need the same amount of translation and overhead as non-native Python code.

If there exist a build-in function for the command you want to use (and you do not have very good reasons to do otherwise) use the build-in function, as:

- Less typing and thinking means less errors in your code.
- Native implementations should give you better performance.

### Arrays

Remember how Python lists could contain different *types* of items?

If we know what type all objects in our collection are, it makes sense to explicitly state the type. Then the Python interpreter can take advantage of the type declaration, which results in faster computations.
**Arrays** are part of the Python Standard Library and provide a collection that is very similar to lists, but specifies the type of contained objects (and thus restricts all contents to be of that type).

We can import the array data structure from the array library with:

In [366]:
from array import array

To create an array data structure need to specify the type that *all* items in the array will have (in this case I pick the type double, *'d'*), and as second argument the collection of elements that are contained in the array:

In [367]:
pythonArray = array('d', long_list)

Notice that we cannot declare arrays with items of different types!

In [368]:
array('d', [1,"this does not work"])

TypeError: a float is required

Let's see if adding an array instead of a list improves performance:

In [369]:
%%timeit
sum(pythonArray)

100 loops, best of 3: 6.29 ms per loop


On the same laptop we didn't get any speed-up compared to the native sum operation! Why is that?
The reason is that Python's native ``sum()`` does operate upon lists. The Python interpreter has to translate our array into a list to perform the operation and therefore we do not get any speedups (in fact we should be slightly slower).

As you can see at the [array documentation](https://docs.python.org/3/library/array.html), the number of operations defined on the Python array implementation is very restricted - and all operations that *are implemented* (like ```reverse``` or ```count```) are already heavily optimized for Python lists.

This was a bit underwhelming, but let's give it one last chance - in the form of NumPy's implementation of arrays and the *additional support for vectorized functions*.

# NumPy 

NumPy is not part of the Python Standard Library, but we installed it in the last lab and if everything was setup correctly (remember starting your conda environment) you can import it using:

In [1]:
import numpy

Before going into more detail on what NumPy is and what the advantages of using it are, let's check how fast the NumPy sum of our list is.

In [379]:
numpyArray = numpy.array(long_list) #we will discuss NumPy array creation in the next section

numpy.sum(numpyArray) #notice that this is NumPys sum implementation

124999750000

In [380]:
%%timeit
numpy.sum(numpyArray)

1000 loops, best of 3: 438 µs per loop


As you can see this is *way faster* than our fastest implementation, and this difference in speed will only increase with the size of our data (If you don't believe it, try a larger range for our long_list).

Why is NumPy so much faster than the Python array? The *NumPy ndarray object is of fixed size* and *all elements are the same datatype* as the Python array. In addition to the array data structure, *Numpy operations are performed as optimized code* on the array data structure.

As you will see in this and the forthcoming labs, changing your iterative (loopy) programs to operate on arrays and use vectorized functions in NumPy can drastically improve performance. Additionally, using Numpy can often result in very short code (and that can make it more readable).

At its core NumPy only provides two main features:
- The [ndarray data sructure](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html), an *n dimensional array*
- [ufuncs](https://docs.scipy.org/doc/numpy/reference/ufuncs.html),  universal functions performed on ndarrays. 


Let's start by having a look at how we can create different forms of arrays in NumPy. 

A NumPy ``ndarray`` is a N-dimensional array, which serves as a container for large arrays of data of the same type. Each ndarray has a **shape** and a **data type** ``dtype``.

## Creating NumPy arrays

The easiest way to create an array is to use the NumPy array function. The array functions accepts any sequence-like object (lists, other arrays, etc.) and produces a new NumPy array containing the data.

In [381]:
numberArray = numpy.array([1,2,3])
numberArray

array([1, 2, 3])

In [382]:
charArray = numpy.array(['a', 'b', 'c'])
charArray

array(['a', 'b', 'c'], 
      dtype='<U1')

In [383]:
floatArray = numpy.array([1, 2, 3.0])
floatArray

array([ 1.,  2.,  3.])

In [384]:
boolArray = numpy.array([True, False])
boolArray

array([ True, False], dtype=bool)

For Python arrays we had to declare the type of the contained elements - here we didn't do any of this, but NumPy has set the type by **inferring the optimal data type when you create an array**.

We can access the data type of a NumPy array with:

In [385]:
boolArray.dtype

dtype('bool')

In [386]:
charArray.dtype

dtype('<U1')

In [387]:
numberArray.dtype

dtype('int64')

In [388]:
floatArray.dtype

dtype('float64')

Notice that this tells you the **type of the contained data**!  If you ask for the type of object ```boolArray``` we instead get ```numpy.ndarray```.

In [389]:
type(boolArray)

numpy.ndarray

We can also include an optional argument to explicitly specify the data type, like so:

In [390]:
numpy.array([1,2,3],dtype='int8')

array([1, 2, 3], dtype=int8)

In [391]:
numpy.array([1,2,3],dtype='uint8')

array([1, 2, 3], dtype=uint8)

<div class="alert alert-info" role="alert">
<h1>Exercises</h1>

<ol>


<li>
Assume you have a large database of user behavior on a video-streaming platform. You want to store different information about the movies in your database in an array. Since your database is very big, the data type in which you store this data will make a big difference in how much space you have to allocate for it on your server. Have a look at the table of different [NumPy types](https://docs.scipy.org/doc/numpy/user/basics.types.html) and think about which data type you would pick for the following data:

<ul style="list-style-type: none;">
   <li><input type="checkbox"> The number of videos watched by each user.</li>
    <li><input type="checkbox"> The average rating of a each movie (ratings from 0 to 10).</li>
    <li><input type="checkbox"> The size of each movie (in MB, in GB?).</li>
    <li><input type="checkbox"> If a user has watched a specific movie.</li>
    <li><input type="checkbox"> The title of the movie that the user has watched.</li>
    <li><input type="checkbox"> The first letter of the title.</li>
</ul>

</li>
<li>
Check what happens if you declare the following NumPy arrays and explain how NumPy handles these arrays (inspecting the resulting array and using the table on [NumPy types](https://docs.scipy.org/doc/numpy/user/basics.types.html).

<ul style="list-style-type: none;">
   <li><input type="checkbox"> numpy.array([-1,2,3],dtype='uint8')</li>
   <li><input type="checkbox"> numpy.array([50,80,250,256])</li>
    <li><input type="checkbox"> numpy.array([50,80,250,256],dtype='float64')</li>
    <li><input type="checkbox"> numpy.array([50,80,250,256],dtype='uint8')</li>
</ul>
</li>
</ol>

</div>

### 2 Dimensional Arrays

We can create a 2-dimensional array  by passing a each row as a a sequence-like object (for example a list). If we want to create a 3x3 array:

In [392]:
 numpy.array([[1,2,3],[4,5,6],[7,8,9]])

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

If instead we want a 2x3 array (2 rows and 3 columns):

In [393]:
numpy.array([[1,2,3],[4,5,6]])

array([[1, 2, 3],
       [4, 5, 6]])

## Shapes and Size of Arrays

One of the most basic property of an array we might be interested in is its *shape*. The NumPy function ``numpy.shape`` returns the shape of an array as a tuple of integers. Each number in the tuple denotes the length of the corresponding array dimension. Let's see an example:

In [394]:
A = numpy.array([1,2,3,4,5,6,7,8,9,10])
A.shape

(10,)

The array A has length 10 in the first dimension (rows) and no other dimensions.

In [395]:
B = numpy.array([[1,2,3],[4,5,6]])
B.shape

(2, 3)

The array B has length 2 in the first dimension (rows) and length 3 in the second dimension (columns).

If instead, we want to know the total number of elements in the array we use ```size```:

In [396]:
A.size

10

In [397]:
B.size

6

In addition to numpy.array, there are several other functions for creating commonly used arrays:

In [398]:
zeroArray = numpy.zeros(2)
zeroArray

array([ 0.,  0.])

In [399]:
oneArray = numpy.ones(3)
oneArray

array([ 1.,  1.,  1.])

In [400]:
sevenArray = numpy.full(12, 7.0)
sevenArray

array([ 7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.])

In [401]:
identityMatrix = numpy.eye(3)
identityMatrix

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

<div class="alert alert-info" role="alert">
<h1>Exercises</h1>
<ol>
<li>
<ul style="list-style-type: none;">
   <li><input type="checkbox"> Create a (3,3) array of ones.</li>
    <li><input type="checkbox"> Create a (7,2) array of zeros.</li>
    <li><input type="checkbox"> Why can't you create a (3,2) identity matrix?</li>
    <li><input type="checkbox"> Create an 1D-array of 25 numbers [0..25] (check numpy.arange).</li>
    <li><input type="checkbox"> Create an 1D-array of 25 evenly spaced numbers between 0,100 (check numpy.linspace).</li>
    <li><input type="checkbox"> What is the difference between arange and linspace?</li>
    <li><input type="checkbox"> What does numpy.logspace do?</li>
    <li><input type="checkbox"> Create a (0:5,0:5) meshgrid (numpy.mgrid).</li>
    <li><input type="checkbox"> Create a (5,5) array of all zeros apart from the diagonal, which is [0,1,2,3,4,5].</li>
    <li><input type="checkbox"> Create a (3,3) array of random values, uniformly distributed between [0,1] (numpy.random).</li>
    
</ul>
</li>
</ol>
</div>

## Indexing and Slicing

Accessing arrays is very similar to accessing Python lists:


In [402]:
testArray = numpy.arange(25)
testArray

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24])

In [403]:
testArray[0]

0

In [404]:
testArray[-1]

24

### Higher dimensional arrays

Higher dimensional arrays consist of an array of one-dimensional arrays, i.e. providing a single index will return the n-th element in the first dimension (which is an array for non 1D-arrays).

In [405]:
testArray2 = numpy.random.rand(3,3)
testArray2

array([[ 0.99041128,  0.90119682,  0.43209712],
       [ 0.9261592 ,  0.25318581,  0.50804226],
       [ 0.95284086,  0.88249747,  0.34941437]])

In [406]:
testArray2[0]

array([ 0.99041128,  0.90119682,  0.43209712])

We can index individual elements in a 2D-array by recursively indexing those 1D-arrays.

For example, if we want the first row and the second column of our array:


In [407]:
testArray2[0][1]

0.9011968162230577

There is another way to access elements of multidimensional arrays in NumPy:

In [408]:
testArray2[1,1]

0.25318581280240104

<div class="alert alert-warning" role="alert">
<h1>Warning</h1>
Accessing the index directly with `Array[row, column]` is more efficient then the nested access, `Array[row][column]`. In the nested case the intermediate array `Array[row]` is created and only then accessed, whereas `Array[row, column]` does not create this intermediate result.
</div>

## Slicing

As for Python lists, NumPy provides **slicing** access for arrays. The we access elements with slicing specifying a start, end and an optional step-size:

In [411]:
testArray[3:5]

array([3, 4])

Notice that the start is *inclusive* and the end is *exclusive* (**from start until end**). If we specify a step-size we can skip elements. 

If we want every third element from the first to the last element in our array:

In [412]:
testArray[0:-1:3]

array([ 0,  3,  6,  9, 12, 15, 18, 21])

We slice multidimensional arrays by specifying the slicing ranges for each dimension separated by a comma, like so:

In [413]:
testArray2 = numpy.array([[1,2,3],[4,5,6],[7,8,9]])
testArray2

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [414]:
testArray2[0:2,0:-1]

array([[1, 2],
       [4, 5]])

In [415]:
testArray2[0:2,0:-1:2]

array([[1],
       [4]])

<div class="alert alert-info" role="alert">
<h1>Exercises</h1>

<ol>
<li>
Slicing is a very important concept for array access. Have a look at the documentation on [Basic Slicing and Indexing](https://docs.scipy.org/doc/numpy/reference/arrays.indexing.html) to find out what the following slices do on an array A:

<ul style="list-style-type: none;">
   <li><input type="checkbox">  A is 1D: A[-3:3:-1]</li>
   <li><input type="checkbox">  A is 1D: A[3:]</li>
   <li><input type="checkbox">  A is 2D: A[1:]</li>
   <li><input type="checkbox">  A is 1D: A[:]</li>
   <li><input type="checkbox">  A is 2D: A[:]</li>
   <li><input type="checkbox">  A is 1D: A[::2]</li>
   <li><input type="checkbox">  A is 2D: A[::2]</li>
   <li><input type="checkbox">  A is 2D: A[::2,::2]</li>
    
</ul>
</li>
</ol>
</div>

<div class="alert alert-warning" role="alert">
<h1>Warning</h1>

Slicing a lists creates a new object (see Lab2), but **slicing an array creates a reference to the original (sub-) array** (in NumPy called a [view](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.view.html)).


This might lead to some confusion, but we can use this to our advantage for modifying arrays efficiently. By selecting a view on our original data and passing it around we can modify the original data by modifying the view (this is beyond this introduction, but if you are curious have a look at the documentation for an [example](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.view.html).
</div>

## Boolean Indexing

With boolean indexing we can select a subset of our array based on a logical condition. For example:

In [285]:
A = numpy.arange(55)
A > 2

array([False, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

As you can see NumPy applies the logical condition (greater than 2) to each element in the array. This works equally for multidimensional arrays:

In [416]:
A = numpy.array([numpy.arange(25),numpy.arange(25)])
A > 2

array([[False, False, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True],
       [False, False, False,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True]], dtype=bool)

We can use the array of type boolean to index subsets of our array like so:


In [417]:
A = numpy.arange(50)
A[A<5]

array([0, 1, 2, 3, 4])

Which is equivalent to:

In [418]:
boolIdx = (A<5)
A[boolIdx]

array([0, 1, 2, 3, 4])

# Computing with Arrays

We have seen how we can create arrays in NumPy and how we can access individual elements, or larger element collections from NumPy arrays. To finish this lab, we will have a quick look at the possibilities that NumPy provides us to perform optimized computations on arrays. 

As the central data structure in NumPy is the array, the computations upon these n-dimensional arrays belong to the field of [Linear Algebra](https://en.wikipedia.org/wiki/Linear_algebra), and NumPy provides implementations for most common operations, e.g. matrix multiplication, decompositions, determinants, etc..

## Scalars

Let's start by performing arithmetic operations on arrays with <a href="https://en.wikipedia.org/wiki/Scalar_(mathematics)">scalars</a>. We can add a scalar to a vector exactly as you would expect:

In [419]:
A = numpy.ones(4)
A

array([ 1.,  1.,  1.,  1.])

In [420]:
A + 0.5

array([ 1.5,  1.5,  1.5,  1.5])

In [421]:
B = numpy.ones([2,2])
B - 0.3

array([[ 0.7,  0.7],
       [ 0.7,  0.7]])

Equally, we can also subtract, divide, multiply  or exponentiate scalars:

In [423]:
A - 0.001 #or A - 1e-3 

array([ 0.999,  0.999,  0.999,  0.999])

In [424]:
A /3

array([ 0.33333333,  0.33333333,  0.33333333,  0.33333333])

In [425]:
2.5 * A

array([ 2.5,  2.5,  2.5,  2.5])

In [426]:
3 ** numpy.arange(15)

array([      1,       3,       9,      27,      81,     243,     729,
          2187,    6561,   19683,   59049,  177147,  531441, 1594323,
       4782969])

## Arithmetic Operations with two Arrays
If instead of a scalar we add,subtract, multiply or divide another array, the operation is performed element-wise:

In [427]:
A = numpy.arange(5)
A

array([0, 1, 2, 3, 4])

In [428]:
B = numpy.arange(5,10)
B

array([5, 6, 7, 8, 9])

We can either use the NumPy function ``add(A,B)``, or the operator overloading ``A+B``.

In [429]:
numpy.add(A,B)

array([ 5,  7,  9, 11, 13])

In [430]:
A + B

array([ 5,  7,  9, 11, 13])

Both return the element-wise addition of the arrays.

In [431]:
R = numpy.random.rand(2,2)
R

array([[ 0.54765584,  0.93434513],
       [ 0.7158221 ,  0.60307349]])

In [432]:
O = numpy.ones([2,2])
O

array([[ 1.,  1.],
       [ 1.,  1.]])

In [433]:
R + O

array([[ 1.54765584,  1.93434513],
       [ 1.7158221 ,  1.60307349]])

Multiplication, subtraction and division are implemented accordingly:

In [434]:
A - B

array([-5, -5, -5, -5, -5])

In [435]:
numpy.subtract(A,B)

array([-5, -5, -5, -5, -5])

In [436]:
A / B

array([ 0.        ,  0.16666667,  0.28571429,  0.375     ,  0.44444444])

In [437]:
numpy.divide(A,B)

array([ 0.        ,  0.16666667,  0.28571429,  0.375     ,  0.44444444])

In [438]:
numpy.multiply(A,B)

array([ 0,  6, 14, 24, 36])

In [439]:
A * B

array([ 0,  6, 14, 24, 36])

<div class="alert alert-warning" role="alert">
<h1>Warning</h1>
Unlike some other languages (like MATLAB), multiplying two two-dimensional arrays with * is an element-wise product in NumPy instead of a matrix dot product.
<br/>
<br/>

<em>Array multiplication is not matrix multiplication!</em>

</div>

## Matrix Multiplication:

If we want to perform [matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication) we can use ```numpy.dot(A,B)```:

In [440]:
A

array([0, 1, 2, 3, 4])

In [441]:
B

array([5, 6, 7, 8, 9])

In [442]:
numpy.dot(A,B)

80

<div class="alert alert-info" role="alert">
<h1>Exercises</h1>

<ol>
<li>
Find the NumPy implementation for the following operations and apply them on our arrays A and/or B:

<ul style="list-style-type: none;">
   <li><input type="checkbox">  Calculate the remainder of two arrays.</li>
   <li><input type="checkbox">  Calculate the Cosine of an array.</li>
   <li><input type="checkbox">  Calculate the natural exponential of an array.</li>
   <li><input type="checkbox">  [Array Transposition](https://en.wikipedia.org/wiki/Transpose)</li>
   <li><input type="checkbox">  [Inner Product](https://en.wikipedia.org/wiki/Matrix_multiplication#The_inner_and_outer_products)</li>
   <li><input type="checkbox">  What is the difference between numpy.inner and numpy.dot?</li>
   <li><input type="checkbox">  [Inverse Matrix](https://en.wikipedia.org/wiki/Invertible_matrix)</li>
   <li><input type="checkbox">  [Determinant of a Matrix](https://en.wikipedia.org/wiki/Determinant)</li>
   <li><input type="checkbox">  Calculate the mean of an array.</li>
   <li><input type="checkbox">  Calculate the standard deviation of an array.</li>
</ul>
</li>
</ol>
</div>



# References

NumPy offers many more operations as optimized computations on multidimensional arrays and is a fundamental tool for anyone interested in doing scientific computing in Python.

If you want to deepen your understanding of NumPy, here are some good starting points:

- [The SciPy NumPy tutorial](http://www.scipy-lectures.org/intro/numpy/index.html)
- [100 Exercises with solutions in Numpy](http://www.labri.fr/perso/nrougier/teaching/numpy.100/)
- [A Tutorial on Numpy implementing the game of life and complex chemical systems](http://www.labri.fr/perso/nrougier/teaching/numpy/numpy.html)

For a visual introduction to Linear Algebra:
- [Essence of Linear Algebra](https://www.youtube.com/watch?v=kjBOesZCoqc)